In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

import re
import emoji

data_train = pd.read_csv("congressional_tweet_training_data.csv",names=None)
data_test= pd.read_csv("congressional_tweet_test_data.csv",names=None)


In [2]:
data_train 

,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D
...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R


In [3]:
data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)

In [4]:
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0
...,...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R,1
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R,1
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D,0
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R,1


In [5]:
data_train.dropna(subset=['full_text'],inplace=True) #First, I drop the rows with NA in the text column
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0
...,...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R,1
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R,1
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D,0
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R,1


In [6]:
data_train=data_train.reset_index(drop=True) #to fix the indices

In [7]:
data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)

In [8]:
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,party_class
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,0
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,0
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,0
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,0
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,0
...,...,...,...,...,...,...,...,...
264995,264995,516,b'We need to #ExtendCHIP before a single child...,ExtendCHIP,223,2017.0,D,0
264996,264996,0,"b""Our #ObamaCare investigation continued today...",ObamaCare,3,2013.0,D,0
264997,264997,1,b'Congratulations to the new #MissTeenUSA Loga...,MissTeenUSA CT,4,2012.0,D,0
264998,264998,2174,b'Speaking of dishonesty. Nothing like being c...,mosen,1168,2018.0,D,0


### BERT for small data

In [9]:
data_train=data_train[0:100]
data_test=data_test[0:25]

X_train = data_train['full_text']
y_train = data_train['party_class']

X_test = data_test['full_text']
y_test = data_test['party_class']

In [10]:
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0
...,...,...,...,...,...,...,...
95,35,"b'In 5 years of DACA, over 780,000 #DREAMers h...",DREAMers SaveDACA,17,2017.0,D,0
96,118,b'Over 90% of gun owners support expanding bac...,OutShoutTheGunLobby,39,2017.0,D,0
97,26,b'Holi is a time to celebrate renewal &amp; a ...,Holi2015,21,2015.0,D,0
98,78,b'Connecticut\xe2\x80\x99s 4th District lost t...,911remembrance,6,2018.0,D,0


In [11]:
data_test

,Id,favorite_count,full_text,hashtags,retweet_count,year,party,party_class
0,0,70,b'#TaxReform improved the playing field for Am...,TaxReform,13,2018.0,D,0
1,1,27,"b'This #NativeWomensEqualPay Day, we recommit ...",NativeWomensEqualPay,11,NaN,D,0
2,2,49,"b""\xe2\x80\x9cI became convinced that our gene...",MeToo ShatteringTheSilence,24,2017.0,D,0
3,3,14,"b'During #NationalAdoptionMonth, we honor the ...",NationalAdoptionMonth,2,2019.0,D,0
4,4,13,b'Happy #AirborneDay to our @USArmy paratroope...,AirborneDay AirborneAllTheWay,7,2018.0,D,0
5,5,73,"b""Proud to join 200+ colleagues in SCOTUS amic...",LGBT,13,2017.0,D,0
6,6,7,b'To the men and women who have been killed in...,PeaceOfficersMemorialDay,2,2018.0,D,0
7,7,2,b'Was happy to be able to tour two Thermo Fish...,Kalamazoo CuresNow MI06,3,2016.0,D,0
8,8,48,b'President Trump knew from day one that #COVI...,COVID19,19,NaN,D,0
9,9,0,b'Yet another Democrat joins me in bipartisan ...,ObamaCare,1,2013.0,D,0


In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [13]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [14]:
y_train.value_counts()

0    52
1    48
Name: party_class, dtype: int64

In [15]:
y_test.value_counts()

0    25
Name: party_class, dtype: int64

In [16]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
4/4 [==============================] - 15s 2s/step - loss: 0.7227 - accuracy: 0.5000
Epoch 2/5
4/4 [==============================] - 9s 2s/step - loss: 0.6843 - accuracy: 0.5700
Epoch 3/5
4/4 [==============================] - 9s 2s/step - loss: 0.6941 - accuracy: 0.5300
Epoch 4/5
4/4 [==============================] - 9s 2s/step - loss: 0.6998 - accuracy: 0.5200
Epoch 5/5
4/4 [==============================] - 9s 2s/step - loss: 0.6896 - accuracy: 0.4800


In [21]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 3s 3s/step - loss: 0.5465 - accuracy: 0.9600


[0.5465327501296997, 0.9599999785423279]

### Sources: 
    
https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/47_BERT_text_classification/BERT_email_classification.ipynb
https://www.youtube.com/watch?v=D9yyt6BfgAM&ab_channel=codebasics